In [ ]:
# la dimensionalità è tutto l'alfabeto, in questo caso possiamo fermarci alla lettera E.
# la dimensionalità dei vettori (rappresentanti le stringhe) dev'essere la stessa.

import math
import numpy as np
from scipy.linalg import norm

a = ['A', 'B', 'C']
b = ['A', 'C', 'D', 'E']

def prebuild(s1, s2):
  alphabet = set(s1).union(set(s2))
  alphabet = sorted(list(alphabet))
  # print(f's1 U s2 = {alphabet}')
  a_vect = [1 if x in a else 0 for x in alphabet]
  b_vect = [1 if x in b else 0 for x in alphabet]
  # print(a_vect)
  # print(b_vect)
  return (alphabet, a_vect, b_vect)

def match_based_sim(s1,s2):
  (alphabet, a_vect, b_vect) = prebuild(s1,s2)
  sum = 0
  for x in range(len(alphabet)):
    sum += 1 if (a_vect[x] == 1 and a_vect[x] == b_vect[x]) else 0
  return sum

def cosine_sim(s1,s2):
  (alphabet, a_vect, b_vect) = prebuild(s1,s2)
  sum = 0
  for x in range(len(alphabet)):
    sum += a_vect[x] * b_vect[x]
  return ( sum / (norm(a_vect) * norm(b_vect)) )

def jaccard_sim(s1, s2):
  # inteso come len(intersezione) / len(unione)
  (alphabet, a_vect, b_vect) = prebuild(s1,s2)
  return len(set(a).intersection(set(b))) / len(alphabet)

print(f'Match-Based: {match_based_sim(a, b)}')
print(f'Cosine: {cosine_sim(a, b)}')
print(f'Jaccard: {jaccard_sim(a, b)}')

Match-Based: 2
Cosine: 0.5773502691896258
Jaccard: 0.4


# Exercise 1

1. Compute the match-based similarity, cosine similarity and the Jaccard coefficient, between the two sets {A,B,C} and {A,C,D,E}.

2. Compute the Edit distance, and LCSS similarity between (a) ababcabc and babcbc (b) cbacbacba and acbacbacb.

3. Compute the cosine measure using the raw frequencies between the following two sentences: (a) “The sly fox jumped over the lazy dog.” and (b) “The dog jumped at the intruder.”


In [ ]:
# Implementazione di Francesco Di Liberti
import numpy as np
def edit(x,y, del_cost, ins_cost, rep_cost):
  m = len(x)+1
  n = len(y) +1
  edit = np.zeros((m, n))
  for i in range(m):
    edit[i][0] = i * del_cost
  for j in range(1, n):
    edit[0][j] = j * ins_cost

  for i in range(1, m):
    for j in range(1, n):
      edit[i, j] = min(edit[i-1, j] + del_cost, edit[i, j-1] + ins_cost, edit[i-1, j-1] + (0 if x[i-1] == y[j-1] else rep_cost))

  return edit[m-1,n-1]

def lcss(x,y):
  m = len(x) + 1
  n = len(y) + 1
  lcss = np.zeros((m,n))

  for i in range(1 ,m):
    for j in range(1, n):
      if(x[i-1] == y[j-1]):
        lcss[i,j] = lcss[i-1, j-1] + 1
      else:
        lcss[i,j] = max(lcss[i-1, j], lcss[i, j-1])

  return lcss[m-1,n-1]

In [ ]:
# 2) Compute the Edit distance, and LCSS similarity between (a) ababcabc and babcbc (b) cbacbacba and acbacbacb.
import numpy as np

a1 = 'ababcabc'
a2 = 'babcbc'
b1 = 'cbacbacba'
b2 = 'acbacbacb'
c1 = 'acbacbacbaa'
c2 = 'acbcbacbc'


# costs
del_cost = 1
ins_cost = 1
rep_cost = 2

def edit_distance(x,y):
  edit = np.zeros((len(x)+1, len(y)+1))
  for i in range(1, len(x)+1):
    edit[i][0] = i * del_cost
  for j in range(1, len(y)+1):
    edit[0][j] = j * ins_cost

  for i in range(1, len(x)+1):
    for j in range(1, len(y)+1):
      # guarda definizione slide
      edit[i, j] = min(
          edit[i-1, j] + del_cost,
          edit[i, j-1] + ins_cost,
          edit[i-1, j-1] + ((x[i-1] != y[j-1]) * rep_cost)
          # ----------------(      Iij       )------------
      )

  return edit[len(x), len(y)]

def lcs(X, Y, m, n):
  if (m == 0 or n == 0):
      return 0;
  elif X[m-1] == Y[n-1]:
      return 1 + lcs(X, Y, m-1, n-1);
  else:
      return max(lcs(X, Y, m, n-1), lcs(X, Y, m-1, n));

print('edit distances')
print(edit_distance(a1, a2)) # fa 2 delete
print(edit_distance(b1, b2)) # fa 1 insert e 1 delete
print(edit_distance(c1, c2))
# correct

print('\nlcss')
print(lcs(a1, a2, len(a1), len(a2))) # fa 2 delete
print(lcs(b1, b2, len(b1) ,len(b2))) # fa 1 insert e 1 delete
print(lcs(c1, c2, len(c1), len(c2)))



edit distances
2.0
2.0
4.0

lcss
6
8
8


In [ ]:
# 3) Compute the cosine measure using the raw frequencies between the following two sentences:
# (a) “The sly fox jumped over the lazy dog.” and
# (b) “The dog jumped at the intruder.”

import numpy as np
from collections import Counter
from scipy.linalg import norm

s1 = 'The sly fox jumped over the lazy dog'
s2 = 'The dog jumped at the intruder'
s3 = 'The fox jumped over the dog'

def cos(s1, s2):
  s1_words = [x.lower() for x in s1.split()]
  s2_words = [x.lower() for x in s2.split()]

  words = set(s1_words).union(set(s2_words))

  s1_freq, s2_freq = Counter(), Counter()

  s1_freq.update(s1_words)
  s2_freq.update(s2_words)

  sum = 0
  for w in words:
    sum += s1_freq[w] * s2_freq[w]

  print(f'cosine distance: {( sum / (norm(list(dict(s1_freq).values())) * norm(list(dict(s2_freq).values()))) )}')
  return

cos(s1, s2)
cos(s2, s3)
cos(s1, s3)

cosine distance: 0.6708203932499369
cosine distance: 0.7499999999999999
cosine distance: 0.8944271909999159


# Exercise 2
Write a function to compute the edit distance.

In [ ]:
# fatto sopra



# Exercise 3
Download the KDD Cup Network Intrusion Data Set for the UCI Machine Learning Repository. Create a data set containing only the categorical attributes. Compute the nearest neighbor for each data point using the (a) Match measure, and (b) Inverse Occurrence Frequency Measure.

http://www.lacascia.it/bd2/_kddcup/task.html

http://www.lacascia.it/bd2/_kddcup/kddcup.names

In [ ]:
import pandas as pd
import numpy as np

# labels=list(range(1, 42))+['class']
labels=list(range(0, 42))

df = pd.read_csv('http://www.lacascia.it/bd2/_kddcup/kddcup.data_10_percent_corrected.csv', names=labels)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [ ]:
# just categorical data
cols = df.columns
numer_cols = df._get_numeric_data().columns
indexes = list(set(cols) - set(numer_cols))
indexes
categ_df = df.iloc[:,indexes]
categ_df


,1,2,3,41
0,tcp,http,SF,normal.
1,tcp,http,SF,normal.
2,tcp,http,SF,normal.
3,tcp,http,SF,normal.
4,tcp,http,SF,normal.
...,...,...,...,...
494016,tcp,http,SF,normal.
494017,tcp,http,SF,normal.
494018,tcp,http,SF,normal.
494019,tcp,http,SF,normal.


In [ ]:
categ_df.iloc[:, 3].value_counts()

In [ ]:
data = categ_df.iloc[:,:3].to_numpy(copy=True)
target = categ_df.iloc[:, -1].to_numpy(copy=True)
print(target.shape, data.shape)

(494021,) (494021, 3)


In [ ]:
# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=7)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Prendo solo 1% del dataset per motivi di tempo di calcolo
X_train = X_train[: round(X_train.shape[0] * 0.001)]
y_train = y_train[: round(y_train.shape[0] * 0.001)]
X_test = X_test[: round(X_test.shape[0] * 0.001)]
y_test = y_test[: round(y_test.shape[0] * 0.001)]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(370515, 3)
(370515,)
(123506, 3)
(123506,)
(371, 3)
(371,)
(124, 3)
(124,)


In [ ]:
def match_based_sim(r1i,r2i):
  sum = 0
  for x in range(categ_df.shape[1]):
    sum += 1 if (categ_df.iloc[r1i, x] == categ_df.iloc[r2i, x]) else 0
  return sum

# print(categ_df.iloc[0,0])
# print(categ_df.shape[1])

match_based_sim(0, 1)

4

In [ ]:
# Calcolo la matrice delle distanze per il train
from scipy.spatial.distance import cosine

dist_train = np.zeros((X_train.shape[0], X_train.shape[0]))
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
    dist_train[i, j] = match_based_sim(i, j)
print(f'dist_train: {dist_train.shape}')

# Calcolo la matrice delle distanze del test dal train
dist_test = np.zeros((X_test.shape[0], X_train.shape[0]))
for i in range(X_test.shape[0]):
  for j in range(X_train.shape[0]):
    dist_train[i, j] = match_based_sim(i, j)
print(f'dist_test: {dist_test.shape}')

# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(X=dist_train, y=y_train)

print(f'Accuracy: {knn.score(dist_test, y_test):.2%}')

dist_train: (371, 371)
dist_test: (124, 371)
Accuracy: 50.00%


# Exercise 4
Repeat Exercise 3 using only the quantitative attributes of the data set, and using the Lp-norm for values of p = 1, 2, ∞.

# Exercise 5
Repeat Exercise 3 using all attributes in the dataset. Use the mixed-attribute function, and different combinations of the categorical and quantitative distance functions of Exercises 3 and 4.


# Exercise 6

IMDB sentiment analysis

distinzione tra commenti positivi e commenti negativi.
Fare un classificatore.
Consiglio prof: usare TF-IDF.

In [2]:
# create stopwords file
import json

sw = {'stopwords': ['call', 'upon', 'still', 'nevertheless', 'down', 'every', 'forty', '‘re', 'always', 'whole', 'side', "n't", 'now', 'however', 'an', 'show', 'least', 'give', 'below', 'did', 'sometimes', 'which', "'s", 'nowhere', 'per', 'hereupon', 'yours', 'she', 'moreover', 'eight', 'somewhere', 'within', 'whereby', 'few', 'has', 'so', 'have', 'for', 'noone', 'top', 'were', 'those', 'thence', 'eleven', 'after', 'no', '’ll', 'others', 'ourselves', 'themselves', 'though', 'that', 'nor', 'just', '’s', 'before', 'had', 'toward', 'another', 'should', 'herself', 'and', 'these', 'such', 'elsewhere', 'further', 'next', 'indeed', 'bottom', 'anyone', 'his', 'each', 'then', 'both', 'became', 'third', 'whom', '‘ve', 'mine', 'take', 'many', 'anywhere', 'to', 'well', 'thereafter', 'besides', 'almost', 'front', 'fifteen', 'towards', 'none', 'be', 'herein', 'two', 'using', 'whatever', 'please', 'perhaps', 'full', 'ca', 'we', 'latterly', 'here', 'therefore', 'us', 'how', 'was', 'made', 'the', 'or', 'may', '’re', 'namely', "'ve", 'anyway', 'amongst', 'used', 'ever', 'of', 'there', 'than', 'why', 'really', 'whither', 'in', 'only', 'wherein', 'last', 'under', 'own', 'therein', 'go', 'seems', '‘m', 'wherever', 'either', 'someone', 'up', 'doing', 'on', 'rather', 'ours', 'again', 'same', 'over', '‘s', 'latter', 'during', 'done', "'re", 'put', "'m", 'much', 'neither', 'among', 'seemed', 'into', 'once', 'my', 'otherwise', 'part', 'everywhere', 'never', 'myself', 'must', 'will', 'am', 'can', 'else', 'although', 'as', 'beyond', 'are', 'too', 'becomes', 'does', 'a', 'everyone', 'but', 'some', 'regarding', '‘ll', 'against', 'throughout', 'yourselves', 'him', "'d", 'it', 'himself', 'whether', 'move', '’m', 'hereafter', 're', 'while', 'whoever', 'your', 'first', 'amount', 'twelve', 'serious', 'other', 'any', 'off', 'seeming', 'four', 'itself', 'nothing', 'beforehand', 'make', 'out', 'very', 'already', 'various', 'until', 'hers', 'they', 'not', 'them', 'where', 'would', 'since', 'everything', 'at', 'together', 'yet', 'more', 'six', 'back', 'with', 'thereupon', 'becoming', 'around', 'due', 'keep', 'somehow', 'n‘t', 'across', 'all', 'when', 'i', 'empty', 'nine', 'five', 'get', 'see', 'been', 'name', 'between', 'hence', 'ten', 'several', 'from', 'whereupon', 'through', 'hereby', "'ll", 'alone', 'something', 'formerly', 'without', 'above', 'onto', 'except', 'enough', 'become', 'behind', '’d', 'its', 'most', 'n’t', 'might', 'whereas', 'anything', 'if', 'her', 'via', 'fifty', 'is', 'thereby', 'twenty', 'often', 'whereafter', 'their', 'also', 'anyhow', 'cannot', 'our', 'could', 'because', 'who', 'beside', 'by', 'whence', 'being', 'meanwhile', 'this', 'afterwards', 'whenever', 'mostly', 'what', 'one', 'nobody', 'seem', 'less', 'do', '‘d', 'say', 'thus', 'unless', 'along', 'yourself', 'former', 'thru', 'he', 'hundred', 'three', 'sixty', 'me', 'sometime', 'whose', 'you', 'quite', '’ve', 'about', 'even']}

with open('stopwords.json', 'w') as jfile:
  json.dump(sw, jfile)

In [3]:
# load stopwords
with open('stopwords.json', 'r') as jfile:
  sw = json.load(jfile)

  stopwords = set([x for x in sw['stopwords']])
  others = set()
  for x in stopwords:
    if (x[0].isalpha() == False):
      others.add(x[1:])

  for y in others:
    stopwords.add(y)

  print(stopwords)

{'these', 'everything', 'will', 'eleven', 'latter', 'give', 'make', 'both', 'others', 'someone', 'that', 'over', 'seemed', 'the', 'being', 'whereas', 'anywhere', 'neither', "'s", "'m", 'at', 'yet', 'seeming', 'thereby', 'why', 'even', 'themselves', 'its', 'here', 'via', 'thus', "'d", 'you', 'without', 'never', 'many', 'which', 'ourselves', 'out', 'upon', 'another', 'several', 'from', 'five', '’ve', 'empty', 'bottom', 'her', 'really', 'anyway', 'such', '’d', 'whereafter', 'part', 'every', 'in', 'could', 'us', 'get', 'wherein', 'made', '’ll', 'because', 'has', 'used', 'on', 'between', 'some', 'namely', 'hence', 'least', 'those', 'take', 'against', 'amongst', 're', 'own', 'i', 'one', 'nowhere', 'no', 'across', 'yourselves', 'herein', 'whole', 'then', 'sixty', 'beforehand', 'during', 'fifteen', 'enough', 'twelve', 'our', 'only', 'under', 'for', 'well', 'am', 'else', 'he', 'mostly', 'often', 'doing', 'whither', 'did', 'four', 'twenty', 'above', 'm', 'may', 'a', 'front', '‘re', 'indeed', 'pl

In [4]:
import pandas as pd
# problema di classificazione
labels=['text', 'class']
df = pd.read_csv('http://www.lacascia.it/bd2/movie.csv', names=labels)
df = df.drop(0)
df.reset_index()
original_df = df
original_df

,text,class
1,I grew up (b. 1965) watching and loving the Th...,0
2,"When I put this movie in my DVD player, and sa...",0
3,Why do people who do not know what a particula...,0
4,Even though I have great interest in Biblical ...,0
5,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39996,"""Western Union"" is something of a forgotten cl...",1
39997,This movie is an incredible piece of work. It ...,1
39998,My wife and I watched this movie because we pl...,0
39999,"When I first watched Flatliners, I was amazed....",1


In [ ]:
df = original_df

In [5]:
alphabet = set()
for index, row in df.iterrows():
    # print(f' index: {index}) -> ({row["text"]}, {row["class"]})')

    words = [x.lower() for x in row["text"].split(' ')]

    # keep just alphabetic words
    fwords = list(filter(lambda x: x.isalpha(), words))

    # remove stopwords
    fwords = list(filter(lambda x: x not in stopwords, fwords))
    set_fwords = set(fwords)

    # fill alphabet
    alphabet = alphabet.union(set_fwords)

    # text column is now made of set of words
    row['text'] = set_fwords

alphabet = sorted(alphabet)
print(len(alphabet))
print(list(alphabet)[:10])
# df

74874
['aa', 'aaa', 'aaaahhhhhh', 'aaaand', 'aada', 'aadha', 'aag', 'aage', 'aah', 'aailiyah']


In [ ]:
print(len(alphabet))
print(list(alphabet)[1234:1244])

74874
['aimable', 'aimants', 'aime', 'aimed', 'aimee', 'aiming', 'aimless', 'aimlessly', 'aims', 'aimé']


In [ ]:
df.iloc[1,0]

In [6]:
import numpy as np
from collections import Counter
from scipy.linalg import norm

def cos(s1, s2, alphabet):
  s1_words = list(s1)
  s2_words = list(s2)

  s1_freq, s2_freq = Counter(), Counter()

  s1_freq.update(s1_words)
  s2_freq.update(s2_words)

  sum = 0
  for w in alphabet:
    sum += s1_freq[w] * s2_freq[w]

  return ( sum / (norm(list(dict(s1_freq).values())) * norm(list(dict(s2_freq).values()))) )

In [7]:
cos(df.iloc[0,0], df.iloc[1,0], alphabet)

dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


0.04845015831115092

In [ ]:
df.iloc[0,0]

In [ ]:
data = df.iloc[:,:1].to_numpy(copy=True)
target = df.iloc[:, -1].to_numpy(copy=True)
print(target.shape, data.shape)
# data[0]

(40000,) (40000, 1)


In [ ]:
# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=7)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Prendo solo 1% del dataset per motivi di tempo di calcolo
X_train = X_train[: round(X_train.shape[0] * 0.005)]
y_train = y_train[: round(y_train.shape[0] * 0.005)]
X_test = X_test[: round(X_test.shape[0] * 0.005)]
y_test = y_test[: round(y_test.shape[0] * 0.005)]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(30000, 1)
(30000,)
(10000, 1)
(10000,)
(150, 1)
(150,)
(50, 1)
(50,)


In [ ]:
# Calcolo la matrice delle distanze per il train
dist_train = np.zeros((X_train.shape[0], X_train.shape[0]))
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
    dist_train[i, j] = cos(df.iloc[i,0], df.iloc[j,0], alphabet)
print(f'dist_train: {dist_train.shape}')

# Calcolo la matrice delle distanze del test dal train
dist_test = np.zeros((X_test.shape[0], X_train.shape[0]))
for i in range(X_test.shape[0]):
  for j in range(X_train.shape[0]):
    dist_train[i, j] = cos(df.iloc[i,0], df.iloc[j,0], alphabet)
print(f'dist_test: {dist_test.shape}')

# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(X=dist_train, y=y_train)

print(f'Accuracy: {knn.score(dist_test, y_test):.2%}')

KeyboardInterrupt: 